## **Installing Libraries**

In [11]:
!pip install python_speech_features

In [12]:
#  os and argparse is done to read files from local folders
import os
import argparse

import numpy as np
from scipy.io import wavfile
import scipy.signal
from python_speech_features import mfcc
import matplotlib.pyplot as plt
import warnings
import random

# tesorflow is for the Deep learning model
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense ,LSTM , TimeDistributed
from tensorflow.keras.layers import Dropout
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

import IPython
import librosa
from scipy.signal import butter, lfilter
%matplotlib inline
import glob

In [13]:
#from google.colab import drive  # the sounds are stored in google drive 
#drive.mount('/content/gdrive')

In [14]:
# folder where files are stored
#cd /content/gdrive/'My Drive'/Colab Notebooks/data/training-a   

**about the dataset**

In [15]:
#############################################################  MFCC EXTRACTION   ##########################################################
# Variable assignment 
tt = 0
time_steps = 450
data_directory=os.getcwd()
print(data_directory)
#data_directory='/workspaces/python_env1/Creative_factory/the-circor-digiscope-phonocardiogram-dataset-1.0.3/AV'
#data_directory=dataset = '../dataset_heart_sound/AV'
data_directory=dataset = '../dataset_heart_sound/AV'
nfft = 1203 # Number of FFTs

# What diff ?
digit_directory = data_directory 

# To normalize the signal
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm
    
def process_directory():
    mfcc_features = list()
    """
    for filename in [x for x in os.listdir(data_directory) if x.endswith('.wav')]:
        # Read the .wav input file
        filepath = os.path.join(digit_directory, filename)
        sampling_freq, audio = wavfile.read(filepath)
        label="n.wav"


# open the .hea file of the same filename to get the label as normal /abnormal.
        st= data_directory +"/"+filename.split(".")[0]+".hea"
        with open(st,'r') as f:
            for line in f:
                for word in line.split():
                    if(word=="Abnormal"):
                        label="a.wav"
"""
    #dataset = [{'sampling_freq', 'audio': wavfile.read(path) , 'label': path.split('/' )[6] } for path in glob.glob("/workspaces/python_env1/Creative_factory/the-circor-digiscope-phonocardiogram-dataset-1.0.3/AV/**/*.wav")]
    for path in glob.glob("../dataset_heart_sound/AV/**/*.wav"):
        sampling_freq, audio = wavfile.read(path)
        label = path.split('/')[3]
            
# now we have the label stored in 'label' and the audio as 'audio' with sampling freq. as 'sampling_freq'.
        #audio1 = audio[dic[filename.split(".")[0]]:]
        #audio1 = SVDnoise(audio/32768)
        audio1 = audio
        temp = mfcc(audio1, sampling_freq, nfft=nfft)
        temp = temp[tt:tt+time_steps,:]
        mfcc_features.append({"label": label, "mfcc": temp })


        # mfcc features of this audio has been appended to the list 
    return mfcc_features

###########################   CREATING MFCC FEATURES   ############################
processed_files = list()
mfcc_features = process_directory()
random.shuffle(mfcc_features)

############   TRAINING DATA   ###########
size = (8*len(mfcc_features))/10
train_features = mfcc_features[0:int(size)]
test_list = mfcc_features[int(size+1):]
train_size = 0
for feature in train_features:
    train_size += 1
    processed_files.append({'label': feature["label"], 'feature': feature["mfcc"] })
# Train rnn for each MFCC and add to training set
x_train = np.zeros((train_size, time_steps ,13))
y_train = np.zeros((train_size))
i = 0
for processed_file in processed_files:
#       print(processed_file['label'])
#       print(processed_file['feature'].shape)
    x_train[i,:,:] = processed_file['feature']
    s = processed_file['label']
    if(s[0]=='a'):
        y_train[i]=1
    else:
        y_train[i]=0
    i += 1
normalize(x_train)

############   TESTING DATA   #############
test_files = list()
test_features = test_list
test_size = 0
for feature in test_features:
    test_size += 1
    test_files.append({'label': feature["label"], 'feature': feature["mfcc"] })
y_test = np.zeros((test_size))
x_test = np.zeros((test_size, time_steps ,13))
i = 0
for test_file in test_files:
    x_test[i,:,:] = test_file['feature']
    s = test_file['label']
#             print(s)
    if(s[0]=='a'):
        y_test[i]=1
    else:
        y_test[i]=0
    i += 1
normalize(x_test)
print('MFCC EXTRACTION DONE !')

/workspaces/Creative_factory_bit_2022/LSTM_demo
MFCC EXTRACTION DONE !


In [27]:
# input size
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_test)
print(np.count_nonzero(y_test == 0))
print(np.count_nonzero(y_train == 0))
print(len(mfcc_features))
print(mfcc_features[1])
print(len(mfcc_features[1]['mfcc']))
print(len(mfcc_features[1]['mfcc'][1]))
print(len(x_train[1][1]))

(629, 450, 13)
(157, 450, 13)
(629,)
(157,)
[1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1.
 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1.
 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1.
 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0.
 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1.]
80
314
787
{'label': 'normal', 'mfcc': array([[ 18.94922505, -19.61940194,  -7.84488204, ...,  -2.70096884,
         -0.20033944,  -1.30123459],
       [ 16.57168818, -15.81586785,   3.56729346, ...,   6.1885533 ,
         -1.57125911,   4.94157173],
       [ 19.24671712, -22.79319255, -10.02837188, ...,  -7.43946962,
         -1.15753791,  -3.39792213],
       ...,
       [ 15.58839344,  26.67785986,  10.23820998, ..., -13.45850704,
          5.13085329,  -7.65847274],
  

## RNN MODEL
### Model Structure:
- visible layer or input layer : size of input = 13 (mfcc matrix has column size 13)
- hidden layer 1: LSTM layer
- hidden layer 2: LSTM layer
- 1 dense layer having activation function = "relu" (rectified linear)
- output layer : classification  

![download.png](attachment:download.png)  
<font size="2">*In the figure, the dense layer is missing, but this is to give a brief idea of how the data is flowing in the model structure.*</font>  
### Building the Model:
- loss function: to compute the loss (currently "mean squared error")
- optimizer function: adam
- metrics: accuracy
- **model.fit:** this function tries for the best possible fit of the model to the training data.
<br>
<font size="2"> <font color="brown"> The later part of the code was to try the model for different values of Dopout(lmabda) to calculate accuracy.</font>


In [17]:
dropouts = np.array([0.15])
accuracy = np.zeros(len(dropouts),dtype=float)

In [18]:
#########################  VARIABLES  ######################
cell_no = 13
Epoch_size = 50
Lambda = 0.029  # dropout variable for regularization
# No. of LSTM layers =2
# Cost func. = cosh

####################   MODEL STRUCTURE  ####################
visible=Input(shape=(None,13))
hidden11 = LSTM(cell_no,return_sequences=True)(visible)
hidden1=Dropout(Lambda)(hidden11)

hidden22 = LSTM(cell_no)(hidden1)
hidden2=Dropout(Lambda)(hidden22)

hidden33 = Dense(10, activation='relu')(hidden2)
hidden3 = Dropout(Lambda)(hidden33)
# hidden4 = TimeDistributed(Dense(1))

output = Dense(1, activation='sigmoid')(hidden3)
# output=Dropout(Lambda)(output1)

model = Model(inputs=visible, outputs=output)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
##################   MODEL END ########################
count = 0
for dr in dropouts:
    total=0
    for ii in range(1):
        Lambda = dr
        print(dr)

        model.fit(x_train, y_train, epochs=Epoch_size, batch_size=10,verbose=1)
        predict=model.predict(x_test)
        y_pred=(predict>0.1)
        acc=model.evaluate(x_test,y_test,verbose=0)
        # print(predict)

        total += acc[1]
        print(acc)
    total /=1
    accuracy[count] = total
    count += 1

0.15
Epoch 1/50
63/63 [==============================] - 12s 92ms/step - loss: 0.2522 - accuracy: 0.4897
Epoch 2/50
63/63 [==============================] - 6s 93ms/step - loss: 0.2452 - accuracy: 0.5628
Epoch 3/50
63/63 [==============================] - 6s 93ms/step - loss: 0.2405 - accuracy: 0.5771
Epoch 4/50
63/63 [==============================] - 6s 97ms/step - loss: 0.2358 - accuracy: 0.6089
Epoch 5/50
63/63 [==============================] - 7s 105ms/step - loss: 0.2268 - accuracy: 0.6216
Epoch 6/50
63/63 [==============================] - 8s 121ms/step - loss: 0.2194 - accuracy: 0.6804
Epoch 7/50
63/63 [==============================] - 6s 93ms/step - loss: 0.2110 - accuracy: 0.6725
Epoch 8/50
63/63 [==============================] - 6s 98ms/step - loss: 0.2003 - accuracy: 0.6916
Epoch 9/50
63/63 [==============================] - 7s 107ms/step - loss: 0.1875 - accuracy: 0.7409
Epoch 10/50
63/63 [==============================] - 8s 134ms/step - loss: 0.1796 - accuracy: 0.7440

In [28]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, 13)]        0         
                                                                 
 lstm_2 (LSTM)               (None, None, 13)          1404      
                                                                 
 dropout_3 (Dropout)         (None, None, 13)          0         
                                                                 
 lstm_3 (LSTM)               (None, 13)                1404      
                                                                 
 dropout_4 (Dropout)         (None, 13)                0         
                                                                 
 dense_2 (Dense)             (None, 10)                140       
                                                                 
 dropout_5 (Dropout)         (None, 10)                0   

In [ ]:

#Print f1, precision, and recall scores
print(precision_score(y_test, y_pred , average="macro"))
print(recall_score(y_test, y_pred , average="macro"))
print(f1_score(y_test, y_pred , average="macro"))
confusion = confusion_matrix(y_test,y_pred,labels=[0,1])
print(confusion)

In [20]:
accuracy

array([0.51592356])